In [2]:
import os
import numpy as np
from tensorflow.python.keras.callbacks import TensorBoard
from time import time
x_files = os.listdir("BIG_DATASET/XDATA")
y_files = os.listdir(f"BIG_DATASET/YDATA")

In [4]:
import tensorflow as tf
from tensorflow.keras.optimizers import *

# Solve OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
# Set up the CPU and GPU devices
strategy = tf.distribute.MirroredStrategy()

with tf.device('/CPU:0'):
    X = np.load("BIG_DATASET/XDATA/" + x_files[0])
    y = np.load(f"BIG_DATASET/YDATA/" + y_files[0], allow_pickle=True)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [3]:
with tf.device('/CPU:0'):

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(5000, 12, 1)),
        tf.keras.layers.Masking(mask_value=-10),
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=2, activation='softmax')
    ])

    # Compile the modelz
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
    #tensorboard = TensorBoard(log_dir="LOGS/{}".format(time()))

In [4]:
import numpy as np
with tf.device('/CPU:0'):
    def data_generator(x_file_list, y_file_list, batch_size):
        while True:
            for x_file, y_file in zip(x_file_list, y_file_list):
                # Load x and y data from numpy files
                x_data = np.load(x_file, allow_pickle=True)
                y_data = np.load(y_file, allow_pickle=True)

                y_data_onehot = tf.keras.utils.to_categorical(y_data, num_classes=2)
                # Create a binary mask for NaN values
                mask = ~np.isnan(x_data)

                # Apply the mask to x_data
                x_data = np.where(mask, x_data, -10)
                
                # Split data into batches
                num_batches = len(x_data) // batch_size
                for i in range(num_batches):
                    batch_x = x_data[i * batch_size : (i + 1) * batch_size]
                    batch_y = y_data_onehot[i * batch_size : (i + 1) * batch_size]
                    yield batch_x, batch_y

In [5]:
train_x_files = os.listdir("BIG_DATASET/XDATA")
train_y_files = os.listdir("BIG_DATASET/YDATA")

In [6]:
for i in range(0, len(train_x_files)):
    train_x_files[i] = "BIG_DATASET/XDATA/" + train_x_files[i]

for i in range(0, len(train_y_files)):
    train_y_files[i] = f"BIG_DATASET/YDATA/" + train_y_files[i]

In [7]:
# X VALIDATION SET
with tf.device('/CPU:0'):
    xval = np.load('BIG_DATASET/XVALIDATE/data-0.npy')
    mask = ~np.isnan(xval)
    xval = np.where(mask, xval, -10)
    for i in range(1, 90):
        x = np.load(f'BIG_DATASET/XVALIDATE/data-{i}.npy')
        mask = ~np.isnan(x)
        x = np.where(mask, x, -10)
        xval = np.concatenate((xval, x), axis=0)
    print("Done loading XVALIDATION files.")

Done loading XVALIDATION files.


In [8]:
# Y VALIDATION SET
with tf.device('/CPU:0'):
    yval = np.load(f'BIG_DATASET/YVALIDATE/data-0.npy')

    for i in range(1, 90):
        y = np.load(f'BIG_DATASET/YVALIDATE/data-{i}.npy')
        yval = np.concatenate((yval, y), axis=0)
    yval = tf.keras.utils.to_categorical(yval, num_classes=2)
    print("Done loading YVALIDATION files.")

Done loading YVALIDATION files.


array([1., 0.], dtype=float32)

In [ ]:
batch_size = 100 # Batch size
with tf.device('/CPU:0'):
    train_generator = data_generator(train_x_files, train_y_files, batch_size)
    steps_per_epoch = len(train_x_files) * len(train_y_files) * 500 // batch_size

with tf.device('/GPU:0'):
    model.fit(train_generator, steps_per_epoch=steps_per_epoch, validation_data=(xval, yval), epochs=1)

15465/40500 [==========>...................] - ETA: 25:44 - loss: 0.2356 - accuracy: 0.9097

In [10]:
model.save('BIG_DATASET/CNNS/CNN_SB.h5')

In [42]:
model = tf.keras.models.load_model("BIG_DATASET/CNNS/CNN_AF_SOFTMAX.h5")

In [43]:
from keras.utils.vis_utils import plot_model
import visualkeras

visualkeras.layered_view(model).show() # display using your system viewer


In [11]:
with tf.device("/CPU:0"):
    xtest = np.load('BIG_DATASET/XTEST/data-0.npy')

    for i in range(1, 90):
        x = np.load(f'BIG_DATASET/XTEST/data-{i}.npy')
        mask = ~np.isnan(x)
        x = np.where(mask, x, -10)
        xtest = np.concatenate((xtest, x), axis=0)

In [12]:
with tf.device("/CPU:0"):
    ytest = np.load(f'BIG_DATASET/YTEST/data-0.npy', allow_pickle=True)

    for i in range(1, 90):
        y = np.load(f'BIG_DATASET/YTEST/data-{i}.npy', allow_pickle=True)
        ytest = np.concatenate((ytest, y), axis=0)
    ytest = tf.keras.utils.to_categorical(ytest, num_classes=2)

In [13]:
with tf.device("/CPU:0"):
    # Evaluate the model on test data
    test_loss, test_acc = model.evaluate(xval, yval)
    print('Test Accuracy:', test_acc)

225/225 [==============================] - 10s 44ms/step - loss: 0.1826 - accuracy: 0.9735
Test Accuracy: 0.9734722375869751


In [67]:
unique,counts = np.unique(yval, return_counts=True)

array([2500], dtype=int64)

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 4998, 10, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2499, 5, 32)      0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 399840)            0         
                                                                 
 dense (Dense)               (None, 32)                12794912  
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 12,795,265
Trainable params: 12,795,265
Non-trainable params: 0
____________________________________________